In [1]:
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import re

## Dataset creation

In [2]:
# import os
# import librosa
# import soundfile as sf

# # Assuming 'output_filename' is the desired filename for the output WAV file
# output_filename = 'long_form_audio/concatenated_output.wav'
# data=pd.read_csv('../MADHAV_task_1/Task3/metadata_test_clean.tsv', sep='\t', header=None, names=['path', 'text', 'num','sr'])
# # List to store audio chunks
# audio_chunks = []
# target=""
# # Iterate over all files in the directory
# for i in range(10):
#     file_path = f"output/audio_noise{i}.wav"

#     # Load the audio file
#     audio, sr = librosa.load(file_path, sr=None)
#     if(target==""):
#         target=data['text'][i]
#     else:
#         target+=" "+data['text'][i]
#     # Append the audio to the list
#     audio_chunks.append(audio)

# # Concatenate all audio chunks
# concatenated_audio = np.concatenate(audio_chunks)

# # Save the concatenated audio to a new WAV file
# sf.write(output_filename, concatenated_audio, samplerate=sr)
# print(target)

In [3]:
%cd /home/suryansh/MADHAV

/home/suryansh/MADHAV


In [4]:
# txt_file_path = "./hindi_data/givenFolderHSp/txt/12janA.txt"
# file=open("./hindi_data/givenFolderHSp/txt/12janA.txt", 'r')
# target=file.readlines()
# target=re.sub(" +", " ", " ".join(target))
# # Now 'hindi_text' contains the contents of the file as a string
# print(target)

## Model algorithm

In [5]:
def split_audio_using_VAD(input_file, pred, chunk_length_sec=15):
    # Load the audio file
    audio, sr = librosa.load(input_file, sr=None)
    timestamps_df=pd.DataFrame()
    timestamps_df['start']=pred['start'].apply(lambda x: x*sr/1000)
    timestamps_df['end']=pred['end'].apply(lambda x: x*sr/1000)
    print(timestamps_df)
    # Calculate the total duration of the audio in samples
    total_samples = len(audio)

    # Calculate the chunk length in samples
    chunk_length_samples = int(sr * chunk_length_sec)
    print(chunk_length_samples)
    # Create an array to store audio chunks
    audio_chunks = []
    start_sample=0
    end_sample=chunk_length_samples
    split_sec=[]
    k=0
    index=0
    for i, row in timestamps_df.iterrows():
        if (start_sample+chunk_length_samples)>=row['end']:
            end_sample=row['end']
        else:
            audio_chunks.append(audio[int(start_sample):int(end_sample)])
            split_sec.append((index, int(start_sample)/sr, int(end_sample)/sr))
            index+=1
            start_sample=end_sample
            end_sample=row['end']
            while(start_sample+chunk_length_samples<row['end']):
                end_sample=start_sample+chunk_length_samples
                audio_chunks.append(audio[int(start_sample):int(end_sample)])
                split_sec.append((index, int(start_sample)/sr, int(end_sample)/sr))
                start_sample=end_sample
                index+=1
                end_sample=row['end']
            

    last_chunk = audio[int(start_sample):int(total_samples)]
    audio_chunks.append(last_chunk)
    split_sec.append((index, start_sample/sr, total_samples/sr))
    split_sec=pd.DataFrame(split_sec, columns=['index', 'start', 'end'])
    return audio_chunks, split_sec

In [6]:
# from fsmnvad import FSMNVad
# # Specify the input audio file
# input_file = "./hindi_data/givenPMS/newGivenPMS/longAudio/april_v.wav"
# vad = FSMNVad()
# segments = vad.segments_offline(input_file)
# df=pd.DataFrame(segments, columns=['start', 'end'])
# df

In [7]:
# !pip install -U funasr_onnx

In [8]:
# !git lfs install
# !git clone https://huggingface.co/funasr/FSMN-VAD

In [9]:
# from funasr_onnx import Fsmn_vad

# model_dir = "./FSMN-VAD"
# model = Fsmn_vad(model_dir, quantize=True)

# input_file = "./hindi_data/givenFolderHSp/fullAudio/12janA.wav"

# result = model(input_file)

In [10]:
# result = np.asarray(result, dtype=np.int32)
# result=result.reshape((result.shape[1],2))
# df=pd.DataFrame(result, columns=['start', 'end'])
# df

In [11]:
# # Split the audio into 15-second chunks with adjustment for the last chunk
# chunks_array, split_sec = split_audio_using_VAD(input_file, df, 7)

# # Now, chunks_array contains the audio chunks as numpy arrays
# for i, chunk in enumerate(chunks_array):
#     print(f"Chunk {i}: start: {split_sec['start'][i]} end: {split_sec['end'][i]}")

In [12]:
# !pip install -q espnet_model_zoo

In [13]:
# #@title Choose English ASR model { run: "auto" }

# lang = 'en'
# fs = 16000 #@param {type:"integer"}
# tag = 'viks66/asr_train_asr_raw_hindi_bpe500' #@param ["Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave", "kamo-naoyuki/librispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_bpe5000_scheduler_confwarmup_steps40000_optim_conflr0.0025_sp_valid.acc.ave"] {type:"string"}

In [14]:
# !git lfs install
# !git clone https://huggingface.co/viks66/asr_train_asr_raw_hindi_bpe500

In [15]:
%cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500

/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500


In [16]:
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


d = ModelDownloader()
# It may takes a while to download and build models
speech2text = Speech2Text(
    "exp/asr_train_asr_raw_hindi_bpe500/config.yaml",
    "exp/asr_train_asr_raw_hindi_bpe500/valid.acc.ave_10best.pth",
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.5,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

In [17]:
# import pandas as pd
# import librosa.display
# from IPython.display import display, Audio
# import matplotlib.pyplot as plt
# import time
# import numpy as np
# import soundfile as sf

# col_names = ['path', 'text', 'time', 'sr']
# preds = ""
# target = ""

# for i, chunk in enumerate(chunks_array):
#     start_time = time.time()
#     speech = np.array([])
#     duration=split_sec['end'][i]*1000-split_sec['start'][i] * 1000
#     for _, row in df.iterrows():
#         start_sample = row['start']-split_sec['start'][i] * 1000
#         end_sample = row['end']-split_sec['start'][i] * 1000
# #         print(start_sample, end_sample, duration)
#         if(start_sample<0 and end_sample<0):
#             continue
#         if(start_sample>duration and end_sample>duration):
#             break
# #         print("Y")
#         speech = np.concatenate([speech, chunk[int(max(0,start_sample))*16:int(min(duration, end_sample))*16]])

#     print(len(speech))
#     nbests = speech2text(speech)
#     if len(speech) != 0:
# #         nbests = speech2text(speech)
#         text, *_ = nbests[0]
# #         output_filename = f'results/output_chunk{i}.wav'
# #         sf.write(output_filename, speech, samplerate=16000)
#         if(preds==""):
#             preds=(text_normalizer(text))
#         else:
#             preds += " " + (text_normalizer(text))

#     print(i, "/500")
#     elapsed_time = time.time() - start_time
#     print(f"Time taken: {elapsed_time:.2f} seconds")


In [18]:
import os
def fetch_filenames(directory_path, num_files=10):
    # Ensure the specified path is a directory
    if not os.path.isdir(directory_path):
        raise ValueError(f"The provided path '{directory_path}' is not a directory.")

    # Get all files in the directory
    all_files = os.listdir(directory_path)

    # Filter out only files (not directories)
    file_names = [file for file in all_files if os.path.isfile(os.path.join(directory_path, file))]

    # Take the first 'num_files' files (or all files if there are fewer than 'num_files')
    selected_files = file_names[:num_files]

    return selected_files

In [19]:
predictions=[]
ground_truth=[]

In [21]:
import pandas as pd
import librosa.display
from IPython.display import display, Audio
import matplotlib.pyplot as plt
import time
import numpy as np
import soundfile as sf
from funasr_onnx import Fsmn_vad
%cd /home/suryansh/MADHAV
directory_path = "./hindi_data/givenFolderHSp/fullAudio"
num_files_to_fetch = 10
file_names = fetch_filenames(directory_path, num_files_to_fetch)
files = ["./hindi_data/givenFolderHSp/fullAudio/"+file_name for file_name in file_names]
txts=["./hindi_data/givenFolderHSp/txt/"+file_name.replace("wav", "txt") for file_name in file_names]
# files=['./hindi_data/givenFolderHSp/fullAudio/12janA.wav', './hindi_data/givenFolderHSp/fullAudio/11janA.wav','./hindi_data/givenFolderPBData/fullAudio/8483_FullAudio.wav','./hindi_data/givenPMS/newGivenPMS/longAudio/april_v.wav' ]
# txts=['./hindi_data/givenFolderHSp/txt/12janA.txt', './hindi_data/givenFolderHSp/txt/11janA.txt','./hindi_data/givenFolderPBData/txt/8483_FullAudio.txt','./hindi_data/givenPMS/newGivenPMS/txt/april_v.txt']
# files=['./hindi_data/givenFolderPBData/fullAudio/17162_FullAudio.wav']
# txts=['./hindi_data/givenFolderPBData/txt/17162_FullAudio.txt']
for txt_file_path, input_file in zip(txts, files):
    %cd /home/suryansh/MADHAV
    file=open(txt_file_path, 'r')
    target=""
    target=file.readlines()
    target=re.sub(" +", " ", " ".join(target))
    # Now 'hindi_text' contains the contents of the file as a string
#     print(target)
    ground_truth.append(target)
#     print(ground_truth)
    model_dir = "./FSMN-VAD"
    model = Fsmn_vad(model_dir, quantize=True)
    result = model(input_file)
    result = np.asarray(result, dtype=np.int32)
    result=result.reshape((result.shape[1],2))
    df=pd.DataFrame(result, columns=['start', 'end'])
    # Split the audio into 15-second chunks with adjustment for the last chunk
    chunks_array, split_sec = split_audio_using_VAD(input_file, df, 7)
    %cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500

    preds = ""
    for i, chunk in enumerate(chunks_array):
        start_time = time.time()
        speech = np.array([])
        duration=split_sec['end'][i]*1000-split_sec['start'][i] * 1000
        for _, row in df.iterrows():
            start_sample = row['start']-split_sec['start'][i] * 1000
            end_sample = row['end']-split_sec['start'][i] * 1000
    #         print(start_sample, end_sample, duration)
            if(start_sample<0 and end_sample<0):
                continue
            if(start_sample>duration and end_sample>duration):
                break
    #         print("Y")
            speech = np.concatenate([speech, chunk[int(max(0,start_sample))*16:int(min(duration, end_sample))*16]])

        print(len(speech))
    #     nbests = speech2text(chunk)
        if len(speech) >= 10000:
            nbests = speech2text(speech)
            text, *_ = nbests[0]
    #         output_filename = f'results/output_chunk{i}.wav'
    #         sf.write(output_filename, speech, samplerate=16000)
            if(preds==""):
                preds=(text_normalizer(text))
            else:
                preds += " " + (text_normalizer(text))

        print(i, "/", len(chunks_array))
        elapsed_time = time.time() - start_time
        print(f"Time taken: {elapsed_time:.2f} seconds")
    predictions.append(preds)
    torch.cuda.empty_cache() 

/home/suryansh/MADHAV
/home/suryansh/MADHAV
         start         end
0      21120.0    981280.0
1     981280.0   1015360.0
2    1031360.0   1369120.0
3    1373600.0   1942240.0
4    1967520.0   2254400.0
5    2271520.0   3231680.0
6    3231680.0   3264000.0
7    3285920.0   4211200.0
8    4230400.0   4785760.0
9    4791040.0   5151200.0
10   5155680.0   5476800.0
11   5482400.0   6442560.0
12   6442560.0   7146880.0
13   7157120.0   7197760.0
14   7202240.0   8162400.0
15   8162400.0   8615360.0
16   8626080.0   8714240.0
17   8718880.0   9218720.0
18   9228480.0   9581760.0
19   9596480.0  10049280.0
20  10060320.0  10412960.0
21  10417440.0  10710240.0
22  10714720.0  10903040.0
23  10911200.0  11531200.0
24  11535840.0  11575840.0
112000
/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500
90880


/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


0 / 115
Time taken: 1.60 seconds
112000
1 / 115
Time taken: 2.23 seconds
112000
2 / 115
Time taken: 2.17 seconds
112000
3 / 115
Time taken: 2.01 seconds
112000
4 / 115
Time taken: 2.03 seconds
112000
5 / 115
Time taken: 1.93 seconds
112000
6 / 115
Time taken: 2.28 seconds
112000
7 / 115
Time taken: 2.08 seconds
85280
8 / 115
Time taken: 1.28 seconds
34080
9 / 115
Time taken: 0.50 seconds
96000
10 / 115
Time taken: 1.93 seconds
112000
11 / 115
Time taken: 2.08 seconds
112000
12 / 115
Time taken: 2.12 seconds
17760
13 / 115
Time taken: 0.28 seconds
107520
14 / 115
Time taken: 1.89 seconds
112000
15 / 115
Time taken: 1.89 seconds
112000
16 / 115
Time taken: 2.00 seconds
112000
17 / 115
Time taken: 2.02 seconds
112000
18 / 115
Time taken: 1.85 seconds
13120
19 / 115
Time taken: 0.18 seconds
86704
20 / 115
Time taken: 1.53 seconds
112000
21 / 115
Time taken: 2.05 seconds
88160
22 / 115
Time taken: 1.58 seconds
94880
23 / 115
Time taken: 1.75 seconds
112000
24 / 115
Time taken: 2.07 seconds


/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


0 / 102
Time taken: 0.96 seconds
0
1 / 102
Time taken: 0.00 seconds
72000
2 / 102
Time taken: 1.30 seconds
44480
3 / 102
Time taken: 0.82 seconds
107520
4 / 102
Time taken: 2.01 seconds
112000
5 / 102
Time taken: 2.28 seconds
112000
6 / 102
Time taken: 2.16 seconds
112000
7 / 102
Time taken: 2.07 seconds
74240
8 / 102
Time taken: 1.50 seconds
20960
9 / 102
Time taken: 0.33 seconds
23520
10 / 102
Time taken: 0.43 seconds
105600
11 / 102
Time taken: 2.10 seconds
112000
12 / 102
Time taken: 2.32 seconds
112000
13 / 102
Time taken: 1.36 seconds
112000
14 / 102
Time taken: 2.30 seconds
112000
15 / 102
Time taken: 1.90 seconds
112000
16 / 102
Time taken: 2.21 seconds
22560
17 / 102
Time taken: 0.39 seconds
100160
18 / 102
Time taken: 2.00 seconds
112000
19 / 102
Time taken: 2.17 seconds
36160
20 / 102
Time taken: 0.63 seconds
107520
21 / 102
Time taken: 2.14 seconds
112000
22 / 102
Time taken: 2.35 seconds
112000
23 / 102
Time taken: 2.50 seconds
112000
24 / 102
Time taken: 2.31 seconds
1120

...

0 / 119
Time taken: 1.04 seconds
101120
1 / 119
Time taken: 2.06 seconds
112000
2 / 119
Time taken: 2.16 seconds
112000
3 / 119
Time taken: 2.09 seconds
65280
4 / 119
Time taken: 1.05 seconds
104000
5 / 119
Time taken: 1.85 seconds
26240
6 / 119
Time taken: 0.46 seconds
104480
7 / 119
Time taken: 1.74 seconds
61600
8 / 119
Time taken: 1.16 seconds
89760
9 / 119
Time taken: 1.83 seconds
66080
10 / 119
Time taken: 1.24 seconds
75360
11 / 119
Time taken: 1.28 seconds
107520
12 / 119
Time taken: 2.06 seconds
104160
13 / 119
Time taken: 2.04 seconds
63520
14 / 119
Time taken: 1.02 seconds
41280
15 / 119
Time taken: 0.77 seconds
85120
16 / 119
Time taken: 1.55 seconds
112000
17 / 119
Time taken: 2.25 seconds
102240
18 / 119
Time taken: 2.08 seconds
107520
19 / 119
Time taken: 2.22 seconds
112000
20 / 119
Time taken: 2.21 seconds
44000
21 / 119
Time taken: 0.71 seconds
105280
22 / 119
Time taken: 2.19 seconds
112000
23 / 119
Time taken: 2.38 seconds
112000
24 / 119
Time taken: 2.37 seconds
92

/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


0 / 118
Time taken: 2.21 seconds
13120
1 / 118
Time taken: 0.18 seconds
107520
2 / 118
Time taken: 2.07 seconds
112000
3 / 118
Time taken: 2.06 seconds
55360
4 / 118
Time taken: 0.88 seconds
107520
5 / 118
Time taken: 1.97 seconds
112000
6 / 118
Time taken: 2.02 seconds
81440
7 / 118
Time taken: 1.37 seconds
89120
8 / 118
Time taken: 1.75 seconds
112000
9 / 118
Time taken: 2.03 seconds
112000
10 / 118
Time taken: 1.89 seconds
111984
11 / 118
Time taken: 1.96 seconds
112000
12 / 118
Time taken: 1.82 seconds
112000
13 / 118
Time taken: 1.69 seconds
112000
14 / 118
Time taken: 1.81 seconds
85280
15 / 118
Time taken: 1.38 seconds
96800
16 / 118
Time taken: 1.76 seconds
112000
17 / 118
Time taken: 2.07 seconds
71360
18 / 118
Time taken: 1.29 seconds
105760
19 / 118
Time taken: 2.01 seconds
112000
20 / 118
Time taken: 1.39 seconds
50720
21 / 118
Time taken: 1.00 seconds
107520
22 / 118
Time taken: 2.04 seconds
88640
23 / 118
Time taken: 1.55 seconds
89760
24 / 118
Time taken: 1.68 seconds
11

/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


0 / 112
Time taken: 0.72 seconds
93280
1 / 112
Time taken: 1.90 seconds
112000
2 / 112
Time taken: 2.29 seconds
70240
3 / 112
Time taken: 1.31 seconds
106560
4 / 112
Time taken: 1.77 seconds
112000
5 / 112
Time taken: 2.36 seconds
112000
6 / 112
Time taken: 2.59 seconds
112000
7 / 112
Time taken: 2.88 seconds
112000
8 / 112
Time taken: 2.63 seconds
112000
9 / 112
Time taken: 2.58 seconds
112000
10 / 112
Time taken: 2.81 seconds
112000
11 / 112
Time taken: 2.49 seconds
69600
12 / 112
Time taken: 1.44 seconds
112000
13 / 112
Time taken: 2.51 seconds
95520
14 / 112
Time taken: 2.22 seconds
102880
15 / 112
Time taken: 2.30 seconds
112000
16 / 112
Time taken: 2.34 seconds
112000
17 / 112
Time taken: 2.34 seconds
112000
18 / 112
Time taken: 2.41 seconds
112000
19 / 112
Time taken: 2.44 seconds
82224
20 / 112
Time taken: 1.50 seconds
65760
21 / 112
Time taken: 1.28 seconds
47840
22 / 112
Time taken: 0.89 seconds
112000
23 / 112
Time taken: 2.68 seconds
13920
24 / 112
Time taken: 0.21 seconds


In [22]:
print(ground_truth)

[' प्रधानमंत्री नरेन्द्र मोदी ने कहा है कि सरकारी कार्यों और योजनाओं की सफलता के लिए सुशासन सबसे महत्वपूर्ण शर्त है और इसी की बदौलत योजनाओं का लाभ अंतिम व्यक्ति तक पहुंचाया जा सकता है बजट उपरांत अंतिम छोर तक पहुंच कोई भी नागरिक पीछे न छूटे विषय पर वीडियो संदेश के माध्यम से वेबिनार को संबोधित करते हुए प्रधानमंत्री ने कहा कि सरकार ऐसा प्रयास कर रही है कि समाज के अंतिम व्यक्ति तक लाभ पहुंचे इस वेबिनार से लास्ट माइल डिलीवरी से जुडे नए विचार नए सुझाव इस मंथन में से वो अमृत निकलेगा जो हमारे दूरदराज के क्षेत्रों में आखिरी छोर पर बैठे हुए हमारे भाई बहन हैं उनके जीवन में बदलाव लाने के लिए हमें टैक्नोलॉजी का सर्वाधिक उपयोग करते हुए ट्रांसपेरेंसी को सुनिश्चित करना है हितधारक सही हो उसको मिलने की व्यवस्था उसके काम आने वाली हो और समय सीमा में मिले प्रधानमंत्री ने जनजातीय समुदायों और देश के ग्रामीण इलाकों में अंतिम व्यक्ति तक पहुंचने के मंत्र पर तत्काल अमल लाए जाने की जरूरत पर जोर दिया उन्होंने कहा कि सरकार देश के जनजातीय लोगों के विकास की हर संभावना पर पूरी ऊर्जा से काम कर रही है और इस वर्ष का केन्

In [23]:
print(predictions)

['प्रधानमंत्री नरेंद्र मोदी ने कहा है कि सरकारी कार्यों और योजनाओं की तपलता की डाटन सब्द महत्वपूर्ण झट है और किसी की बदालत योजनाओं कला अंतिम व्यक्ति तक पहुँचाया जाता है बजरू ब्रांड अंतिम छोटा बहुत कोई भी नागरिक भी खेलाखूट वीके पर वीडियो संदीप के माध्यम थे वैविनार को संबोधित करते हुए प्रधानमंत्री ने कहा कि सरकार ऐसा प्रयोग कर रही है कि तमाह के अंतिम व्यक्ति टटलापुर के बैबीदार थे लाख मैक डिलीव्र के जुड़े नए विकार नए सुझाव उपमंथन मेथे पूर्व अमृत निकलेगा ग्लोमरिडूर बनाते थे आखिरी छोर पर बैठे भी हमारे फाइबल हैं उनकी जीवन में बदलाव लाने के लिए हम टेक्नोलॉजी का सर्वाधिक उपयोग करते हुए ट्रांसप्रंथी को सुनिश्चित करना हितधारक भी उसको मिलने की व्यवस्था उसके काम आने वाली और हम एक टीमों में मिले प्रधानमंत्री देर द्विजाटीयी समुदायों और ठीक के ग्रामीण इलाकों में अंतिम व्यक्ति तक पहुंचने के बाद पर तत्काल अमल लाये जाने की जरूर पारी रोकियां उलगाह के प्रकार देश के गिरदार की लोगों विका की हर संभावना पर पूरी ऊलिया की कम्प रही है और इटवर्ड का केंद्रीय बजट जनजातीय विकास की यह इसका वर्पित है प्रधानमंत्री ने क

In [24]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(predictions, ground_truth))
wer = WordErrorRate()
print("WER:", wer(predictions, ground_truth))

CER: tensor(0.6331)
WER: tensor(0.8236)


In [26]:
for i in range(10):
    cer = CharErrorRate()
    print("CER:", cer(predictions[i], ground_truth[i]))
for i in range(10):
    wer = WordErrorRate()
    print("WER:", wer(predictions[i], ground_truth[i]))

CER: tensor(0.3179)
CER: tensor(0.4606)
CER: tensor(0.5690)
CER: tensor(0.1471)
CER: tensor(0.1774)
CER: tensor(0.7357)
CER: tensor(0.7746)
CER: tensor(0.7511)
CER: tensor(0.7742)
CER: tensor(0.8046)
WER: tensor(0.5655)
WER: tensor(0.6649)
WER: tensor(0.7215)
WER: tensor(0.3247)
WER: tensor(0.3843)
WER: tensor(0.9318)
WER: tensor(0.9481)
WER: tensor(0.9394)
WER: tensor(0.9496)
WER: tensor(0.9970)


In [25]:
txts

['./hindi_data/givenFolderHSp/txt/27_feb_23.txt',
 './hindi_data/givenFolderHSp/txt/10janA.txt',
 './hindi_data/givenFolderHSp/txt/17janA.txt',
 './hindi_data/givenFolderHSp/txt/12janA.txt',
 './hindi_data/givenFolderHSp/txt/2023_04_14.txt',
 './hindi_data/givenFolderHSp/txt/2023_04_16.txt',
 './hindi_data/givenFolderHSp/txt/14janA.txt',
 './hindi_data/givenFolderHSp/txt/2023_04_09.txt',
 './hindi_data/givenFolderHSp/txt/28_feb_23.txt',
 './hindi_data/givenFolderHSp/txt/17_mar_noon.txt']

In [24]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(predictions[2], ground_truth[2]))
wer = WordErrorRate()
print("WER:", wer(predictions[2], ground_truth[2]))

CER: tensor(0.1637)
WER: tensor(0.3511)


In [25]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(predictions[3], ground_truth[3]))
wer = WordErrorRate()
print("WER:", wer(predictions[3], ground_truth[3]))

CER: tensor(0.7743)
WER: tensor(0.9241)


In [82]:
%cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500

/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500


In [83]:
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


d = ModelDownloader()
# It may takes a while to download and build models
speech2text = Speech2Text(
    "exp/asr_train_asr_raw_hindi_bpe500/config.yaml",
    "exp/asr_train_asr_raw_hindi_bpe500/valid.acc.ave_10best.pth",
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.5,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

In [84]:
import pandas as pd
import soundfile
import librosa.display
from IPython.display import display, Audio
import matplotlib.pyplot as plt
import time

col_names=['path', 'text', 'time', 'sr', 'std']
egs = pd.read_csv("/home/suryansh/MADHAV/hindi_data/givenFolderHSp/tsv/12janA.tsv", sep="\t", names=col_names, header=None, encoding='utf-8', index_col=False)
# print(egs['path'])
preds=[]
target=[]
for index, row in egs.iterrows():
    start_time = time.time()
    speech, rate = soundfile.read("/home/suryansh/MADHAV/hindi_data/givenFolderHSp/finalSplitAudio/" + row["path"])
#     assert fs == int(row["sr"])
    nbests = speech2text(speech)

    text, *_ = nbests[0]
# Uncomment to see details of each prediction
#     print(f"Input Speech: ESPNet_asr_egs/{row['path']}")
    # let us listen to samples
#     display(Audio(speech, rate=rate))
#     librosa.display.waveshow(speech, sr=rate, color="blue")
#     plt.show()
#     print(f"Reference text: {text_normalizer(row['text'])}")
    target.append(text_normalizer(row['text']))
#     print(f"ASR hypothesis: {text_normalizer(text)}")
    preds.append(text_normalizer(text))
    print(index,"/500")
    elapsed_time = time.time() - start_time
    print(f"Time taken: {elapsed_time:.2f} seconds")

/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


0 /500
Time taken: 5.14 seconds
1 /500
Time taken: 5.37 seconds
2 /500
Time taken: 4.41 seconds
3 /500
Time taken: 2.48 seconds
4 /500
Time taken: 4.52 seconds
5 /500
Time taken: 4.49 seconds
6 /500
Time taken: 5.41 seconds
7 /500
Time taken: 5.35 seconds
8 /500
Time taken: 5.75 seconds
9 /500
Time taken: 5.21 seconds
10 /500
Time taken: 4.13 seconds
11 /500
Time taken: 5.15 seconds
12 /500
Time taken: 6.05 seconds
13 /500
Time taken: 5.67 seconds
14 /500
Time taken: 6.45 seconds
15 /500
Time taken: 5.18 seconds
16 /500
Time taken: 5.57 seconds
17 /500
Time taken: 5.97 seconds
18 /500
Time taken: 2.56 seconds
19 /500
Time taken: 5.08 seconds
20 /500
Time taken: 5.53 seconds
21 /500
Time taken: 2.00 seconds
22 /500
Time taken: 4.25 seconds
23 /500
Time taken: 5.51 seconds
24 /500
Time taken: 6.48 seconds
25 /500
Time taken: 5.60 seconds
26 /500
Time taken: 4.99 seconds
27 /500
Time taken: 5.11 seconds
28 /500
Time taken: 5.03 seconds
29 /500
Time taken: 5.10 seconds
30 /500
Time taken: 

In [85]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(preds, target))
wer = WordErrorRate()
print("WER:", wer(preds, target))

CER: tensor(0.1677)
WER: tensor(0.3445)
